TODO:
- Figure out how to properly implement rebalancing
- Add user input reading to determine parameters

#### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import seaborn as sns
import math
import pandas_market_calendars as mcal

### Parameters for the portfolio

Dataframe of 100 most popular etfs for reference

In [7]:
df_etfs = pd.read_excel('ETFList.xlsx')

### Parameters to be used for the test

- Tickers of products to invest in along with the weight

In [28]:
products = {
          "SPY":0.4,    #SP500
          "QQQ":0.2,   #NASDAQ100
          "IWM":0.0,    #Russel2000
           #Bonds
          "BIL":0.0,     #0-3 month 
          "SHY":0.4,    #1-3 yr 
          "IEF":0.0,      #7-10yr
          "TLT":0.0       #20+yr
}

- Start and end dates

In [27]:
start_date = '2010-01-01'
end_date = '2023-01-01'

- Deposit frequency and amount 
    - Possible frequencies: ["once","daily","weekly","monthly","yearly"]

In [32]:
# ['B':business day,'BM':business month end, 'BMS':business month start
frequency = 'BMS' # Deposit frequency
deposit = 100 

Rebalancing

In [33]:
rebalancing_frequency = 'BYS' # How many times a year
rebalancing = True

## Backtesting

Generate dates

In [38]:
# Generate initial deposit dates and put them in a dataframe
deposit_dates = pd.date_range(start_date, end_date)
df = pd.DataFrame({'Dates': deposit_dates})
df["Deposit"] = True


Exclude Market Holidays

In [39]:
# Create a calendar of market dates for NYSE
nyse = mcal.get_calendar('NYSE')

# Function to find the next trading date given a date
def next_trading_day(date):
    while True:
        date = date + pd.Timedelta(days=1) # check the following date
        if nyse.valid_days(start_date=date, end_date=date).size > 0:
            return date


# Replace dates that are US market holidays or weekend days with the nearest trading day
df['Dates'] = df['Dates'].apply(lambda x: next_trading_day(x)
    if nyse.valid_days(start_date=x, end_date=x).size == 0 # Date is a market holiday
    or 
    x.weekday() >= 5 # The weekend
    else x) # Valid Date


In [47]:
df.Dates.drop_duplicates(inplace=True) # Remove duplicates

In [50]:
df = df.drop_duplicates(subset=['Dates'], keep='first') # Remove duplicates

In [51]:
df

,Dates,Deposit
0,2010-01-04,True
4,2010-01-05,True
5,2010-01-06,True
6,2010-01-07,True
7,2010-01-08,True
...,...,...
4740,2022-12-27,True
4744,2022-12-28,True
4745,2022-12-29,True
4746,2022-12-30,True


In [36]:

# Extra check on modified dates: Check if each date in the "Dates" column is a US market holiday or a weekend day
df['Holiday_or_Weekend'] = df['Dates'].apply(lambda x: 'Holiday' if nyse.valid_days(start_date=x, end_date=x).size == 0 else 'Weekend' if x.weekday() >= 5 else 'Trading Day')



In [ ]:

# Add rebalancing dates based on specified rebalancing frequency

# Create a date range with the specified rebalancing frequency
rebalance_dates = pd.date_range(start_date, end_date, freq=rebalancing_frequency).tolist() 
df_rebalancing = pd.DataFrame({'Dates': rebalance_dates}) 

# Create a new column "Rebalancing" in df and initialize it to False
df['Rebalancing'] = False

# Loop through each date in df_rebalancing and mark the corresponding date in df as rebalancing - Very slow
for date in df_rebalancing['Dates']:
    # Find the index of the matching date in df, or the index of the closest date if no match exists
    idx = df['Dates'].sub(date).abs().idxmin()
    # Set the corresponding row in df to be a rebalancing date
    df.loc[idx, 'Rebalancing'] = True


Add ticker info

In [13]:
# Add each desired ticker with its weight to the dataframe

# Create a list of lists with the ticker and its weight
tickers = []
for key, value in products.items():
    if value != 0:
        tickers.append([key, value])


# The below columns represent how many $ have been spent on each ticker at each date
for ticker in tickers:
    col_name = f"{ticker[0]}_spent"
    df[col_name] = ticker[1] * deposit


# The below columns represent how much each ticker cost at each date
for ticker in tickers:
    ticker_price_column = f"{ticker[0]}_price"
    df[ticker_price_column ] = 0
    
    # Get historical prices for our given dates from yahoo finance
    ticker_obj = yf.Ticker(ticker[0])
    history = ticker_obj.history(start=start_date, end=end_date) 
    
    date_list = df['Dates'].dt.strftime('%Y-%m-%d') # get our list of investment dates
    prices = history.loc[date_list]['Close'] # If this line gives an error some of our dates aren't market days
    # TODO: Add catch error line

    df[ticker_price_column] = prices.to_list()





In [14]:
# Add column to indicate number of shares bought at each date, cumulative number of shares bought at each date, 
for ticker in tickers:
    
    new_shares_bought = f"{ticker[0]}_bought"
    df[new_shares_bought] = df[f"{ticker[0]}_spent"] / df[f"{ticker[0]}_price"]

    shares_bought_cumulative = f"{ticker[0]}_bought_cum"
    df[shares_bought_cumulative] = df[new_shares_bought].cumsum()

    shares_value_cumulative = f"{ticker[0]}_value_cum"
    df[shares_value_cumulative] = df[shares_bought_cumulative] * df[f"{ticker[0]}_price"]


In [16]:
df

,Dates,Deposit,Holiday_or_Weekend,Rebalancing,SPY_spent,QQQ_spent,SHY_spent,SPY_price,QQQ_price,SHY_price,SPY_bought,SPY_bought_cum,SPY_value_cum,QQQ_bought,QQQ_bought_cum,QQQ_value_cum,SHY_bought,SHY_bought_cum,SHY_value_cum
0,2010-01-04,True,Trading Day,True,40.0,20.0,40.0,87.791786,40.949291,72.963066,0.455623,0.455623,40.000000,0.488409,0.488409,20.000000,0.548223,0.548223,40.000000
1,2010-02-01,True,Trading Day,False,40.0,20.0,40.0,84.484001,38.161701,73.414490,0.473462,0.929086,78.492895,0.524086,1.012495,38.638516,0.544852,1.093074,80.247481
2,2010-03-01,True,Trading Day,False,40.0,20.0,40.0,86.676277,40.058334,73.550743,0.461487,1.390573,120.529708,0.499272,1.511766,60.558847,0.543842,1.636916,120.396416
3,2010-04-01,True,Trading Day,False,40.0,20.0,40.0,91.630280,42.529583,73.339523,0.436537,1.827110,167.418611,0.470261,1.982027,84.294798,0.545409,2.182325,160.050667
4,2010-05-03,True,Trading Day,False,40.0,20.0,40.0,93.613792,44.092648,73.513885,0.427287,2.254398,211.042708,0.453590,2.435618,107.392835,0.544115,2.726440,200.431181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2022-08-01,True,Trading Day,False,40.0,20.0,40.0,404.170013,313.038116,80.952324,0.098968,35.784473,14463.011062,0.063890,32.516313,10178.845328,0.494118,78.138005,6325.453119
152,2022-09-01,True,Trading Day,False,40.0,20.0,40.0,390.050568,297.280487,80.239914,0.102551,35.887024,13997.754130,0.067277,32.583589,9686.465321,0.498505,78.636510,6309.786824
153,2022-10-03,True,Trading Day,False,40.0,20.0,40.0,362.201294,272.081696,79.515228,0.110436,35.997460,13038.326574,0.073507,32.657097,8885.398246,0.503048,79.139559,6292.800072
154,2022-11-01,True,Trading Day,False,40.0,20.0,40.0,379.895905,273.653320,79.163879,0.105292,36.102752,13715.287610,0.073085,32.730182,8956.722947,0.505281,79.644840,6304.994477


### Implement Rebalancing

In [15]:
# Implement rebalancing



In [18]:
# Add column to indicate cumulative number of shares bought at each date, along with present value of each ticker
for ticker in tickers:
    new_shares_bought = f"{ticker[0]}_shares_bought"
    
    #df[new_ticker_value] = df[shares_bought_cumulative] * df[f"{ticker[0]}_price"]

In [19]:
# Get prices for our given dates from yahoo finance
ticker_obj = yf.Ticker('SHY')
history = ticker_obj.history(start=start_date, end=end_date) # get historical prices

date_list = df['Dates'].dt.strftime('%Y-%m-%d') # get our list of investment dates

prices = history.loc[date_list]['Close'] # If this line gives an error some of our dates aren't market days

In [20]:
prices.to_list()

[72.96306610107422,
 73.41448211669922,
 73.5507583618164,
 73.33951568603516,
 73.51387786865234,
 73.92033386230469,
 74.205322265625,
 74.35635375976562,
 74.50658416748047,
 74.65258026123047,
 74.83583068847656,
 74.53804016113281,
 74.50984191894531,
 74.54348754882812,
 74.57987213134766,
 74.49727630615234,
 74.8228530883789,
 75.15755462646484,
 75.07827758789062,
 75.29313659667969,
 75.60009002685547,
 75.51884460449219,
 75.53491973876953,
 75.56616973876953,
 75.55636596679688,
 75.63775634765625,
 75.54373168945312,
 75.51691436767578,
 75.64949798583984,
 75.7014389038086,
 75.64408111572266,
 75.77497100830078,
 75.78755187988281,
 75.80730438232422,
 75.73546600341797,
 75.78038024902344,
 75.77232360839844,
 75.79116821289062,
 75.8611831665039,
 75.87734985351562,
 75.93756866455078,
 75.84503173828125,
 75.77937316894531,
 75.84054565429688,
 75.76679229736328,
 75.94483947753906,
 75.98894500732422,
 76.04199981689453,
 75.9889144897461,
 76.12664031982422,
 76.178

### Single function to build dataframe with required parameters

In [24]:
def create_df(products,start_date,end_date,frequency,deposit,rebalancing,rebalancing_frequency):
    # Get data from yahoo finance

    # Get list of tickers to use
    tickers_use = sorted([key for key,value in products.items() if value != 0])
    
    #dates = pd.date_range(start_date, end_date,frequency='B')

    data = yf.download(tickers_use, start=start_date, end=end_date)
    df = pd.DataFrame(data['Adj Close'])
    df.reset_index(inplace=True)

    # Add Transact, Deposit, Rebalancing Flags
    df['Deposit'] = 0
    df['Transact'] = False
    df['Rebalance'] = False


    for i in range(0,len(df),frequency):
        df.at[i,'Transact'] = True
        df.at[i,'Deposit'] = deposit

    for i in range(rebalancing_frequency,len(df),rebalancing_frequency):
        df.at[i,'Rebalance'] = True


    #Calculate amount of each instrument bought on given date
    units_bought = [f'{ticker}_units_bought' for ticker in tickers_use ]
    units_cum = [f'{ticker}_units_cum' for ticker in tickers_use ]
    units_value = [f'{ticker}_value' for ticker in tickers_use ]

    for i in range(len(units_bought)): # For each product
        p = tickers_use[i]
        weight = products[p] 
        df[units_bought[i]] =  df.Deposit * weight / df[tickers_use[i]]

        df[units_cum[i]] = df[units_bought[i]].cumsum()

        weight = products[tickers_use[i]] 
        df[units_value[i]] =  df[tickers_use[i]] * df[units_cum[i]]
        
        #if df['Rebalance'][i] == True: ## Commence rebalancing 

    #Portfolio value and cumulative deposits
    df['Portfolio_Value'] = df[units_value].sum(axis=1)
    df['Deposits_cum'] = df.Deposit.cumsum()
    df['OverallPnL%'] = (df['Portfolio_Value'] - df['Deposits_cum'])/ df['Deposits_cum']
    #Figure out smart way to do this
    df['DoDPnL%'] = 0
    df['DoDLogReturn'] = 0
    for i in range(1,len(df)):
        df.at[i,'DoDPnL%'] = (df.at[i,'Portfolio_Value'] - df.at[i-1,'Portfolio_Value']-df.at[i,'Deposit'])/df.at[i-1,'Portfolio_Value']
        df['DoDLogReturn'] =  np.log((df.at[i,'Portfolio_Value'] -df.at[i,'Deposit']) / (df.at[i-1,'Portfolio_Value']-df.at[i,'Deposit']))
    
    return df
    

In [25]:
df = create_df(products,start_date,end_date,frequency,deposit,rebalancing,rebalancing_frequency)

[*********************100%***********************]  3 of 3 completed


TypeError: 'str' object cannot be interpreted as an integer

In [6]:
df.tail()

,Date,SHY,SPY,Deposit,Transact,Rebalance,SHY_units_bought,SHY_units_cum,SHY_value,SPY_units_bought,SPY_units_cum,SPY_value,Portfolio_Value,Deposits_cum,OverallPnL%,DoDPnL%,DoDLogReturn
2511,2019-12-24,82.794525,307.631134,0,False,False,0.00000,183.802342,15217.827627,0.000000,142.526135,43845.476665,59063.304292,35900,0.645217,0.000054,0.00177
2512,2019-12-26,82.823921,309.268768,0,False,False,0.00000,183.802342,15223.230691,0.000000,142.526135,44078.882349,59302.113041,35900,0.651869,0.004043,0.00177
2513,2019-12-27,82.882713,309.192047,100,True,False,0.48261,184.284952,15274.036819,0.194054,142.720190,44127.947575,59401.984394,36000,0.650055,-0.000002,0.00177
2514,2019-12-30,82.912071,307.487457,0,False,False,0.00000,184.284952,15279.447040,0.000000,142.720190,43884.668189,59164.115229,36000,0.643448,-0.004004,0.00177
2515,2019-12-31,82.902283,308.234436,0,False,False,0.00000,184.284952,15277.643165,0.000000,142.720190,43991.277139,59268.920304,36000,0.646359,0.001771,0.00177


### Calculate Performance Metrics

Using Portfolio log returns

In [7]:
r = df.DoDLogReturn.to_numpy()
std = np.sqrt(np.sum(np.square(r)))
std /= np.sqrt(251)
std

0.0056034824620875565

#### Portfolio Volatility as weighted sum of individual securities
- This assumes the portfolio weights hold up

Plotting and stuff